In [17]:
import indoor_pb2

# https://ss1.bdstatic.com/8bo_dTSlR1gBo1vgoIiO_jowehsv/pvd/?qt=vtile&param=5J9FK5%3E%3EKH8:E9DA;I82M5@;EL9FJED;EE@CNHJK=E9GJ8J;EF%3EGN9:L=?%3ENPE2;EE%3E?BHJK=D9FK4%3EOCO82N5B;EG%3ECL5L%3ECB86NE2%3EKD86E8FNMA?JPE43
# 'x=26362&y=7067&z=19&styles=pl&textimg=0&scaler=2&v=088&udt=20190321&json=1'

# from importlib import reload
# reload(map_pb2)

bb = open('/tmp/indoor.bin', 'rb').read()

# bo = map_pb2.BuildingOutline()
indoor = indoor_pb2.Block()

indoor.ParseFromString(bb)

136267

In [29]:
indoor.indoor[1].meta

some_id: "5087719067573934800"
default_floor: "F1"
floors: "B2"
floors: "B1"
floors: "M"
floors: "F1"
floors: "F2"
floors: "F3"
floors: "F4"
floors: "F5"
floors: "F6"
name: "\350\231\271\346\241\245\345\244\251\345\234\260\\\350\264\255\347\211\251\344\270\255\345\277\203"
border {
  points: 26620
  points: 64324
  points: 7961
  points: 305
  points: 5785
  points: 241
  points: 3373
  points: 149
  points: 12813
  points: 469
  points: 1381
  points: 165
  points: 981
  points: 473
  points: 889
  points: 837
  points: 401
  points: 937
  points: 137
  points: 1037
  points: 1
  points: 1037
  points: 88
  points: 1917
  points: 216
  points: 6269
  points: 304
  points: 8289
  points: 5004
  points: 80
  points: 1400
  points: 12
  points: 3512
  points: 72
  points: 208
  points: 6429
  points: 881
  points: 53
  points: 9029
  points: 421
  points: 216
  points: 6681
  points: 220
  points: 5805
  points: 216
  points: 5749
  points: 220
  points: 6269
  points: 120
  points: 2841

In [4]:
indoor.indoor[0]

floor {
  name: "B1"
  idx_num: -1
  b1 {
    b2 {
      some_id: 3
      b3 {
        some_id_b3: 18
        b4 {
          poi {
            x: 24984
            y: 44709
            attr_3: 192
            attr_4: -1095303928646557055
            attr_5: 20000
            name: "\344\270\207\350\276\276\345\271\277\345\234\272"
            attr_10: 1
          }
        }
      }
      b3 {
        some_id_b3: 57
        b4 {
          poi {
            x: 9529
            y: 47890
            attr_3: 252
            attr_4: 8701571035674691892
            attr_5: 19500
            name: "\345\201\234\350\275\246\345\234\272\345\207\272\345\205\245\345\217\243"
            attr_10: 1
          }
        }
        b4 {
          poi {
            x: 42592
            y: 45105
            attr_3: 252
            attr_4: 4824279672282102495
            attr_5: 19500
            name: "\345\201\234\350\275\246\345\234\272\345\207\272\345\205\245\345\217\243"
            attr_10: 1
     

In [3]:
aa = indoor.indoor

In [57]:
13508096>>9, 3579392 >>9

(26383, 6991)

In [18]:
hex(bb.find(aa)), len(aa)

('0x3b95', 117764)

In [53]:
def lastbit_sign(a):
    if a&1:
        return (a>>1)*-1
    else:
        return a>>1

def pg(a, base_x=0, base_y=0, scale=1):
    x0, y0, a = a[0], a[1], a[2:]
    ret = [(x0,y0)]
    while a:
        dx, dy, a = a[0], a[1], a[2:]
        ret.append((ret[-1][0]+dx, ret[-1][1]+dy))
#     if ret[-1][0] - x0 < 0.1 and ret[-1][1] - y0 < 0.1:
#         ret[-1] = x0, y0
    ret = [(base_x+x/scale, base_y+y/scale) for (x, y) in ret]
    return ret

def pb_ints_2_points(ps, base_xy, scale=1):
    ps = [lastbit_sign(i) for i in ps]
    return pg(ps, base_xy[0], base_xy[1], scale)
    
def f_indoor(block, base_xy):
    for indoor in block.indoor:
        indoor_info = {}
        indoor_info['name'] = indoor.meta.name
        indoor_info['floor_names'] = list(indoor.meta.floors)
        indoor_info['border'] = pb_ints_2_points(indoor.meta.border.points, base_xy, 200)
        indoor_info['floors'] = []
        for floor in indoor.floor:
            floor_info = {}
            floor_info['name'] = floor.name
            floor_info['pois'] = []
            floor_info['polygons'] = []
            for b1 in floor.b1:
                for b2 in b1.b2:
                    for b3 in b2.b3:
                        for b4 in b3.b4:
                            for poi in b4.poi:
                                poi_info = {}
                                poi_info['loc'] = (base_xy[0]+poi.x/200, base_xy[1]+poi.y/200)
                                poi_info['tag_3'] = poi.attr_3
                                poi_info['name'] = poi.name
                                floor_info['pois'].append(poi_info)
                            for polygon in b4.b50:
                                polygon_info = {}
#                                 polygon_info['points'] = list(polygon.points)
#                                 ps = [lastbit_sign(i) for i in polygon.points]
                                polygon_info['points'] = pb_ints_2_points(polygon.points, base_xy, 200)
                                polygon_info['tag'] = b3.some_id_b3
                                floor_info['polygons'].append(polygon_info)
            indoor_info['floors'].append(floor_info)
        print(indoor_info)

In [54]:
f_indoor(indoor, (0, 0))

{'name': '万达广场', 'floor_names': ['B1', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6'], 'border': [(172.2, 519.7), (125.93, 519.61), (89.49, 519.23), (89.44, 519.22), (84.13, 519.33), (79.77, 519.28), (65.13, 519.28), (65.14, 518.92), (64.35, 518.91), (65.89, 451.37), (66.09, 446.61), (66.11, 445.81), (66.44, 429.9), (66.44, 427.31), (67.55, 412.42), (135.72, 400.55), (137.85, 400.16), (138.85, 400.01), (135.72, 380.41), (136.27, 380.31), (136.27, 380.27), (160.19, 375.97), (165.96, 374.94), (212.04, 366.53), (253.45, 359.36), (256.73, 358.78), (267.13, 356.92), (275.73, 355.35), (285.23, 353.64), (295.28, 351.92), (301.65, 350.78), (310.59, 349.21), (312.6, 348.78), (327.91, 346.07), (331.56, 365.63), (393.52, 354.13), (406.78, 351.51), (406.92, 352.39), (407.57, 352.28), (409.58, 366.34), (410.81, 374.34), (412.72, 387.24), (412.62, 389.11), (412.94, 391.25), (412.83, 401.08), (412.5, 411.69), (412.23, 416.96), (412.13, 420.48), (412.06, 426.31), (411.92, 428.18), (410.95, 463.96), (391.86, 479.

In [14]:
indoor.indoor[0].floor[2]

name: "F2"
idx_num: 1
b1 {
  b2 {
    some_id: 3
    b3 {
      some_id_b3: 18
      b4 {
        poi {
          x: 24984
          y: 44709
          attr_3: 192
          attr_4: -1095303928646557055
          attr_5: 20000
          name: "\344\270\207\350\276\276\345\271\277\345\234\272"
          attr_10: 1
        }
      }
    }
    b3 {
      some_id_b3: 158
      b4 {
        poi {
          x: 33228
          y: 38389
          attr_3: 60
          attr_4: -8680634576902036949
          attr_5: 18500
          name: "\345\222\225\345\231\234\345\222\225\345\231\234\350\277\220\345\212\250\\\346\210\220\351\225\277\344\270\255\345\277\203"
          attr_10: 1
        }
      }
      b4 {
        poi {
          x: 37082
          y: 38611
          attr_3: 60
          attr_4: 1220215644499862545
          attr_5: 18500
          name: "CICI\347\272\244\347\276\216"
          attr_10: 1
        }
      }
      b4 {
        poi {
          x: 33574
          y: 39211
        

In [9]:
indoor.indoor[0].floor[2].b1[0].b2[1].b3[8].b4[2].b50[0]

attr_1: 248
arr: 127200
arr: 179932
arr: 49
arr: 105
arr: 1609
arr: 3801
arr: 1252
arr: 77
arr: 1520
arr: 3752
arr: 1117
arr: 228

In [8]:
indoor.indoor[0].floor[2].b1[0].b2[0].b3[6]

some_id_b3: 20682
b4 {
  poi {
    x: 26660
    y: 45560
    attr_3: 252
    attr_4: 7783638558930129077
    attr_5: 15960
    name: "zippo"
    attr_10: 1
  }
}

In [3]:
import hexdump
hexdump.hexdump(indoor.indoor[0].meta[0])

00000000: 0A 14 31 37 33 35 31 34  34 30 31 34 35 30 36 32  ..17351440145062
00000010: 39 39 34 35 36 31 12 02  46 31 1A CE 07 12 CB 07  994561..F1......
00000020: B8 9E 02 9C C4 0F C1 03  DD 13 F5 13 C8 03 E5 02  ................
00000030: 3C 95 03 DD 11 89 10 DC  02 E9 02 DD 0F D8 24 DD  <.............$.
00000040: 06 C1 03 E1 13 01 95 0E  58 9D 16 2C A1 16 58 9D  ........X..,..X.
00000050: 16 58 A1 16 58 A1 16 2C  E9 15 58 A1 16 58 A1 16  .X..X..,..X..X..
00000060: 58 A1 16 2C 9D 16 58 A1  16 2C A1 05 95 12 3D 2C  X..,..X..,....=,
00000070: 91 0A 2C 8D 0A 20 95 08  E5 0B 11 E1 2D 01 04 91  ..,.. ......-...
00000080: 01 BD 02 05 E8 04 89 D3  01 50 F1 0E 08 C1 02 84  .........P......
00000090: 01 DD 31 01 8D 08 BC 03  C5 2E D4 34 99 09 A9 01  ..1........4....
000000A0: C5 09 E1 01 D5 09 95 03  C9 12 80 16 E9 02 90 0A  ................
000000B0: E5 01 E4 09 E9 01 90 0A  B1 01 E4 09 E9 01 90 0A  ................
000000C0: E5 01 E4 09 E9 01 90 0A  E5 01 E4 09 E9 01 90 0A  ................

In [3]:
navi.m1.m25.m253

'内江公园北一门 上海市杨浦区控江路261'

In [6]:
88/83, 5303/4985, 5303/60, 4985/60

(1.0602409638554218, 1.063791374122367, 88.38333333333334, 83.08333333333333)

In [8]:
print(MessageToJson(navi))

{
  "m1": [
    {
      "m22": [
        2
      ],
      "m23": "\u4e0a\u7406\u5c0f\u533a(\u4e1c\u95e8) \u4e0a\u6d77\u5e02\u6768\u6d66\u533a\u4e0a\u7406\u5357\u8def",
      "m24": "\u6ef4\u6c34\u6e56 \u4e0a\u6d77\u5e02\u6d66\u4e1c\u65b0\u533a\u73af\u6e56\u897f\u4e00\u8def380\u53f7",
      "m25": {
        "m251": [
          ""
        ],
        "m252": [
          ""
        ],
        "m253": [
          "\u4e0a\u7406\u5c0f\u533a(\u4e1c\u95e8) \u4e0a\u6d77\u5e02\u6768\u6d66\u533a\u4e0a\u7406\u5357\u8def"
        ],
        "m255": [
          27064350,
          7298410
        ],
        "m257": [
          ""
        ]
      },
      "m26": {
        "m251": [
          ""
        ],
        "m252": [
          ""
        ],
        "m253": [
          "\u6ef4\u6c34\u6e56 \u4e0a\u6d77\u5e02\u6d66\u4e1c\u65b0\u533a\u73af\u6e56\u897f\u4e00\u8def380\u53f7"
        ],
        "m255": [
          27150526,
          7196144
        ],
        "m256": [
          0
        ],
        "

In [2]:
from google.protobuf.json_format import MessageToJson
import json
json.loads(MessageToJson(navi))

{'m1': [{'m22': [2],
   'm23': '上理小区(东门) 上海市杨浦区上理南路',
   'm24': '滴水湖 上海市浦东新区环湖西一路380号',
   'm25': {'m251': [''],
    'm252': [''],
    'm253': ['上理小区(东门) 上海市杨浦区上理南路'],
    'm255': [27064350, 7298410],
    'm257': ['']},
   'm26': {'m251': [''],
    'm252': [''],
    'm253': ['滴水湖 上海市浦东新区环湖西一路380号'],
    'm255': [27150526, 7196144],
    'm256': [0],
    'm258': ['']},
   'm28': [1],
   'm2a': [1],
   'm2c': [0],
   'm2d': [0],
   'm2e': [''],
   'm2f': ['']}],
 'm2': [{'m21': [{'m211': [''],
     'm212': ['"g":"6_1","w":"AAAA","p":"8","s":"0","seq":"2","v":"137594971"'],
     'm33': [{'m331': [72867],
       'm332': [5303],
       'm43': [{'m431': [42], 'm432': [0]}],
       'm44': ['']}],
     'm35': [''],
     'm36': [10],
     'm37': [22],
     'm38': [''],
     'm3a': ['Ml8z'],
     'm3d': ['较快'],
     'm3e': [''],
     'm3f': [''],
     'm3g': ['红绿灯<font color="#3385ff">22</font>个'],
     'm3h': ['aceaf60138c6d6afe2a803604082756b137594971'],
     'm3j': [''],
     'm3k': [2]},
    

In [4]:
import google.protobuf
dir(google.protobuf)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'descriptor',
 'descriptor_database',
 'descriptor_pb2',
 'descriptor_pool',
 'internal',
 'json_format',
 'message',
 'message_factory',
 'pyext',
 'reflection',
 'symbol_database',
 'text_encoding']